### Init ray

In [ ]:
import ray
ray.init(address="auto", ignore_reinit_error=True)

### Load default config

In [ ]:
# load defaulft config
import yaml
import os
import time
import pandas as pd
import random

config_path = './configs/AL_cifar.yml'

with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)

# create base dir and gr
if os.path.exists(config["PROJECT"]["project_dir"]) is False:
    os.mkdir(config["PROJECT"]["project_dir"])

if os.path.exists(config["PROJECT"]["group_dir"]) is False:
    os.mkdir(config["PROJECT"]["group_dir"])

### Get the data to annotate

For DataNet we should pass the list of the annotated images

In [ ]:
# Get the data to annotate

#############################################################################################
# LOAD DATA
#############################################################################################
from data_utils import CIFAR10Data
# Load data
cifar10_data = CIFAR10Data()
num_classes = len(cifar10_data.classes)
x_train, y_train, x_test, y_test = cifar10_data.get_data(subtract_mean=True)

In [ ]:
indices = list(range(len(x_train)))
random.shuffle(indices)
labeled_set = indices
unlabeled_set = []

In [ ]:
# test with all the images
NUM_IMAGES_TEST = len(x_test)
# Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
test_set = list(range(NUM_IMAGES_TEST))

In [ ]:
config["NETWORK"]["INPUT_SIZE"] =  x_train[0].shape[0]
config["NETWORK"]["CLASSES"] = cifar10_data.classes

### Set Configuration from the data

In [ ]:
from train_agent_cifar import Active_Learning_train
#from inference_agent_cifar import Active_Learning_inference

In [ ]:
NetworkActor =  Active_Learning_train.remote(config, labeled_set, test_set,  0, False)
NetworkActor.start_training.remote()


In [ ]:
#NetworkActor.__ray_terminate__.remote()
    

In [ ]:
#NetworkActor

In [ ]:
NetworkActor

In [ ]:
progress_id = NetworkActor.isTraining.remote()
response = ray.get(progress_id)